In [ ]:
SET MY_USER = CURRENT_USER();

SET GITHUB_SECRET_USERNAME = 'username';
SET GITHUB_SECRET_PASSWORD = 'password';
SET GITHUB_URL_PREFIX = 'https://github.com/username';
SET GITHUB_REPO_ORIGIN = 'https://github.com/username/sfguide-data-engineering-with-notebooks.git';

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the account level objects (ACCOUNTADMIN part)
-- ----------------------------------------------------------------------------

USE ROLE ACCOUNTADMIN;

-- Roles
CREATE OR REPLACE ROLE USDSE_ROLE;
GRANT ROLE USDSE_ROLE TO ROLE SYSADMIN;
GRANT ROLE USDSE_ROLE TO USER IDENTIFIER($MY_USER);

GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE USDSE_ROLE;
GRANT EXECUTE TASK ON ACCOUNT TO ROLE USDSE_ROLE;
GRANT EXECUTE MANAGED TASK ON ACCOUNT TO ROLE USDSE_ROLE;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE USDSE_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE TO ROLE USDSE_ROLE;

-- Databases
CREATE OR REPLACE DATABASE USD_SPOT_EXCHANGE;
GRANT OWNERSHIP ON DATABASE USD_SPOT_EXCHANGE TO ROLE USDSE_ROLE;

-- Warehouses
CREATE OR REPLACE WAREHOUSE USDSE_WH WAREHOUSE_SIZE = XSMALL, AUTO_SUSPEND = 300, AUTO_RESUME= TRUE;
GRANT OWNERSHIP ON WAREHOUSE USDSE_WH TO ROLE USDSE_ROLE;

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the database level objects
-- ----------------------------------------------------------------------------
-- USE ROLE DEMO_ROLE;
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE USDSE_WH;
USE DATABASE USD_SPOT_EXCHANGE;

-- Schemas
CREATE OR REPLACE SCHEMA INTEGRATIONS;
CREATE OR REPLACE SCHEMA DEV_SCHEMA;
CREATE OR REPLACE SCHEMA PROD_SCHEMA;

USE SCHEMA INTEGRATIONS;

CREATE OR REPLACE STAGE USDSE_RAW_STAGE
    URL = 's3://dexus-team3/'
;

CREATE OR REPLACE FILE FORMAT CSV_FORMAT
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    SKIP_HEADER = 1
    NULL_IF = ('NULL', 'null', '')
    EMPTY_FIELD_AS_NULL = TRUE;

-- Secrets (schema level)
CREATE OR REPLACE SECRET USDSE_GITHUB_SECRET
  TYPE = password
  USERNAME = $GITHUB_SECRET_USERNAME
  PASSWORD = $GITHUB_SECRET_PASSWORD;

-- API Integration (account level)
-- This depends on the schema level secret!
CREATE OR REPLACE API INTEGRATION USDSE_GITHUB_API_INTEGRATION
  API_PROVIDER = GIT_HTTPS_API
  API_ALLOWED_PREFIXES = ($GITHUB_URL_PREFIX)
  ALLOWED_AUTHENTICATION_SECRETS = (USDSE_GITHUB_SECRET)
  ENABLED = TRUE;

-- Git Repository
CREATE OR REPLACE GIT REPOSITORY USDSE_GIT_REPO
  API_INTEGRATION = USDSE_GITHUB_API_INTEGRATION
  GIT_CREDENTIALS = USDSE_GITHUB_SECRET
  ORIGIN = $GITHUB_REPO_ORIGIN;

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the event table
-- ----------------------------------------------------------------------------
USE ROLE ACCOUNTADMIN;

CREATE EVENT TABLE USD_SPOT_EXCHANGE.INTEGRATIONS.USDSE_EVENTS;
GRANT SELECT ON EVENT TABLE USD_SPOT_EXCHANGE.INTEGRATIONS.USDSE_EVENTS TO ROLE USDSE_ROLE;
GRANT INSERT ON EVENT TABLE USD_SPOT_EXCHANGE.INTEGRATIONS.USDSE_EVENTS TO ROLE USDSE_ROLE;

ALTER ACCOUNT SET EVENT_TABLE = USD_SPOT_EXCHANGE.INTEGRATIONS.USDSE_EVENTS;
ALTER DATABASE USD_SPOT_EXCHANGE SET LOG_LEVEL = INFO;

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE USDSE_WH;
USE SCHEMA USD_SPOT_EXCHANGE.INTEGRATIONS;

EXECUTE IMMEDIATE FROM @USDSE_GIT_REPO/branches/main/scripts/deploy_notebooks.sql
    USING (env => 'DEV', branch => 'dev');